In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time

### DL all short video url from a channel url

In [2]:
def get_youtube_shorts_urls(channel_url):
    """
    指定されたYouTubeチャンネルのShortsタブにある動画URLを全て取得する関数
    """
    # Chromeドライバーの初期化
    driver = webdriver.Chrome()
    
    try:
        # Shortsタブへのアクセス
        shorts_url = f"{channel_url}"
        driver.get(shorts_url)
        
        # ページが読み込まれるまで待機
        time.sleep(3)
        
        # スクロールして全ての動画を読み込む
        last_height = driver.execute_script("return document.documentElement.scrollHeight")
        while True:
            # スクロール
            driver.execute_script("window.scrollTo(0, document.documentElement.scrollHeight);")
            time.sleep(2)
            
            # 新しい高さを取得
            new_height = driver.execute_script("return document.documentElement.scrollHeight")
            if new_height == last_height:
                break
            last_height = new_height
        
        # 動画のURLを取得
        video_elements = driver.find_elements(By.CSS_SELECTOR, "a[title]")
        video_urls = [elem.get_attribute('href') for elem in video_elements 
                     if elem.get_attribute('href') and '/shorts/' in elem.get_attribute('href')]
        return video_urls
        
    finally:
        driver.quit()

# Shorts URLの取得
channel_url = "https://www.youtube.com/@Hmminds/shorts"
shorts_urls = get_youtube_shorts_urls(channel_url)
print(f"取得したShortsの数: {len(shorts_urls)}")
for url in shorts_urls:
    print(url)


取得したShortsの数: 0


### DL videos from video urls (1080x1920 only)

In [26]:
from pytube import YouTube
import os

from yt_dlp import YoutubeDL

def download_youtube_short(url, output_path='videos'):
    """
    YouTube Shortの動画をダウンロードする関数
    1080x1920の解像度を持つフォーマットを選択してダウンロード
    """
    ydl_opts_info = {
        'quiet': True,
        'no_warnings': True,
    }
    
    try:
        with YoutubeDL(ydl_opts_info) as ydl:
            info = ydl.extract_info(url, download=False)
            formats = info['formats']
            
            # 1080x1920の解像度を持つフォーマットを探す
            target_formats = [f for f in formats 
                            if f.get('width') == 1080 
                            and f.get('height') == 1920 
                            and f.get('vcodec') != 'none'
                            and not f.get('protocol', '').startswith('m3u8')]  # m3u8プロトコルを除外
            
            if not target_formats:
                raise Exception('1080x1920の解像度を持つ適切なフォーマットが見つかりませんでした')
            
            # FPSが最も高いものを選択
            best_format = max(target_formats, key=lambda f: (f.get('fps', 0)))
            format_id = best_format['format_id']
            
            print(f'選択されたフォーマットID: {format_id} '
                  f'(解像度: {best_format.get("width")}x{best_format.get("height")}, '
                  f'FPS: {best_format.get("fps", "不明")})')
            
            # ダウンロード用のオプション
            ydl_opts = {
                'format': format_id,  # 選択したビデオフォーマットのみ
                'outtmpl': f'{output_path}/%(title)s.%(ext)s',
                'quiet': False,
                'no_warnings': False,
            }
            
            # ダウンロードを実行
            with YoutubeDL(ydl_opts) as ydl:
                ydl.download([url])
            
            # ダウンロードした動画をvideosディレクトリに移動
            if not os.path.exists(output_path):
                os.makedirs(output_path)
            
            # ダウンロードした動画のファイル名を取得
            video_filename = os.path.basename(ydl.prepare_filename(info))
            src_path = ydl.prepare_filename(info)
            dst_path = os.path.join(output_path, video_filename)
            
            # shutilを使って移動(既存ファイルは上書き)
            import shutil
            shutil.move(src_path, dst_path)

        print('ダウンロードが完了しました')
    except Exception as e:
        print(f'エラーが発生しました: {str(e)}')

for url in shorts_urls:
    download_youtube_short(url, output_path='videos')


選択されたフォーマットID: 299 (解像度: 1080x1920, FPS: 60)
[youtube] Extracting URL: https://www.youtube.com/shorts/021sJGsf68E
[youtube] 021sJGsf68E: Downloading webpage
[youtube] 021sJGsf68E: Downloading tv client config
[youtube] 021sJGsf68E: Downloading player 1080ef44
[youtube] 021sJGsf68E: Downloading tv player API JSON
[youtube] 021sJGsf68E: Downloading ios player API JSON
[youtube] 021sJGsf68E: Downloading m3u8 information
[info] 021sJGsf68E: Downloading 1 format(s): 299
[download] Destination: videos\Poor Cat Living in The Landfill Strives to Become Rich After Being Mocked #hmminds.mp4
[download] 100% of   31.25MiB in 00:00:05 at 5.84MiB/s     
ダウンロードが完了しました
選択されたフォーマットID: 299 (解像度: 1080x1920, FPS: 60)
[youtube] Extracting URL: https://www.youtube.com/shorts/KHNJI1y221Q
[youtube] KHNJI1y221Q: Downloading webpage
[youtube] KHNJI1y221Q: Downloading tv client config
[youtube] KHNJI1y221Q: Downloading player 37364e28
[youtube] KHNJI1y221Q: Downloading tv player API JSON
[youtube] KHNJI1y221Q: D

### DL a video from a video url

In [3]:
from pytube import YouTube
import os

from yt_dlp import YoutubeDL

def download_youtube_short(url, output_path='videos'):
    """
    YouTube Shortの動画をダウンロードする関数
    1080x1920の解像度を持つフォーマットを選択してダウンロード
    """
    ydl_opts_info = {
        'quiet': True,
        'no_warnings': True,
    }
    
    try:
        with YoutubeDL(ydl_opts_info) as ydl:
            info = ydl.extract_info(url, download=False)
            formats = info['formats']
            
            # 1080x1920の解像度を持つフォーマットを探す
            target_formats = [f for f in formats 
                            if f.get('width') == 1080 
                            and f.get('height') == 1920 
                            and f.get('vcodec') != 'none'
                            and not f.get('protocol', '').startswith('m3u8')]  # m3u8プロトコルを除外
            
            if not target_formats:
                raise Exception('1080x1920の解像度を持つ適切なフォーマットが見つかりませんでした')
            
            # FPSが最も高いものを選択
            best_format = max(target_formats, key=lambda f: (f.get('fps', 0)))
            format_id = best_format['format_id']
            
            print(f'選択されたフォーマットID: {format_id} '
                  f'(解像度: {best_format.get("width")}x{best_format.get("height")}, '
                  f'FPS: {best_format.get("fps", "不明")})')
            
            # ダウンロード用のオプション
            ydl_opts = {
                'format': format_id,  # 選択したビデオフォーマットのみ
                'outtmpl': f'{output_path}/%(title)s.%(ext)s',
                'quiet': False,
                'no_warnings': False,
            }
            
            # ダウンロードを実行
            with YoutubeDL(ydl_opts) as ydl:
                ydl.download([url])
            
            # ダウンロードした動画をvideosディレクトリに移動
            if not os.path.exists(output_path):
                os.makedirs(output_path)
            
            # ダウンロードした動画のファイル名を取得
            video_filename = os.path.basename(ydl.prepare_filename(info))
            src_path = ydl.prepare_filename(info)
            dst_path = os.path.join(output_path, video_filename)
            
            # shutilを使って移動(既存ファイルは上書き)
            import shutil
            shutil.move(src_path, dst_path)

        print('ダウンロードが完了しました')
    except Exception as e:
        print(f'エラーが発生しました: {str(e)}')

url = "https://www.youtube.com/shorts/DdFZ1yLo3jw"
download_youtube_short(url, output_path='videos')

選択されたフォーマットID: 137 (解像度: 1080x1920, FPS: 30)
[youtube] Extracting URL: https://www.youtube.com/shorts/DdFZ1yLo3jw
[youtube] DdFZ1yLo3jw: Downloading webpage
[youtube] DdFZ1yLo3jw: Downloading tv client config
[youtube] DdFZ1yLo3jw: Downloading player 7795af42
[youtube] DdFZ1yLo3jw: Downloading tv player API JSON
[youtube] DdFZ1yLo3jw: Downloading ios player API JSON
[youtube] DdFZ1yLo3jw: Downloading m3u8 information
[info] DdFZ1yLo3jw: Downloading 1 format(s): 137
[download] Destination: videos\Transformation into minecraft chracters ｜ Spiderman vs Sonic vs Deadpool #spiderman #joker #sonic.mp4
[download] 100% of   17.06MiB in 00:00:01 at 9.06MiB/s   
ダウンロードが完了しました


### Screen shots from videos

In [4]:
import cv2
import numpy as np
import re
import os


def video_to_images(video_path, output_dir='images', interval_sec=0.6):
    """
    動画を一定間隔で画像として保存する関数
    
    Parameters:
    -----------
    video_path : str
        入力動画のパス
    output_dir : str
        出力画像を保存するディレクトリ
    interval_sec : int
        画像を抽出する間隔(秒)
    """
    
    # 出力ディレクトリの作成
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
        
    # 既存の画像ファイルから最大の連番を取得
    existing_files = [f for f in os.listdir(output_dir) if f.startswith('frame_') and f.endswith('.png')]
    max_number = -1
    if existing_files:
        numbers = [int(re.search(r'frame_(\d{4})\.png', f).group(1)) for f in existing_files]
        max_number = max(numbers)
    saved_count = max_number + 1
        
    # 動画の読み込み
    cap = cv2.VideoCapture(video_path)
    
    # フレームレートの取得
    fps = cap.get(cv2.CAP_PROP_FPS)
    frame_interval = int(fps * interval_sec)
    
    frame_count = 0
    
    while cap.isOpened():
        ret, frame = cap.read()
        
        if not ret:
            break
            
        # 指定した間隔でフレームを保存
        if frame_count % frame_interval == 0:
            output_path = os.path.join(output_dir, f'frame_{saved_count:04d}.png')
            cv2.imwrite(output_path, frame)
            saved_count += 1
            
        frame_count += 1
    
    cap.release()
    print(f'{saved_count - (max_number + 1)}枚の画像を保存しました')

# ダウンロードしたファイルのパスを指定して実行
video_files = [f for f in os.listdir('videos') if f.endswith(('.mp4', '.webm'))]
if video_files:
    for video_file in video_files:
        video_path = os.path.join('videos', video_file)
        print(f'処理中: {video_file}')
        video_to_images(video_path)
else:
    print('動画ファイルが見つかりません')


処理中: Transformation into minecraft chracters ｜ Spiderman vs Sonic vs Deadpool #spiderman #joker #sonic.mp4
90枚の画像を保存しました


### Crop images

In [5]:
def crop_images():
    # 出力ディレクトリの作成
    output_dir = 'images_cropped'
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
        
    # imagesフォルダ内の画像ファイルを取得
    image_files = [f for f in os.listdir('images') if f.endswith(('.png', '.jpg', '.jpeg'))]
    
    for image_file in image_files:
        try:
            # 画像の読み込み
            image_path = os.path.join('images', image_file)
            img = cv2.imread(image_path)
            
            if img is None:
                print(f'画像の読み込みに失敗しました: {image_file}')
                continue
                
            # 画像の高さと幅を取得
            height, width = img.shape[:2]
            
            # 上下400pxを除去
            start_y = 300
            end_y = height - 300
            
            # 画像の高さが800px未満の場合はスキップ
            if height < 800:
                print(f'画像が小さすぎるためスキップします: {image_file}')
                continue
                
            # 画像をクロップ
            cropped_img = img[start_y:end_y, :]
            
            # クロップした画像を保存
            output_path = os.path.join(output_dir, image_file)
            cv2.imwrite(output_path, cropped_img)
            
        except Exception as e:
            print(f'エラーが発生しました({image_file}): {str(e)}')
    
    print(f'{len(image_files)}枚の画像をクロップしました')

# クロップ処理の実行
crop_images()


40枚の画像をクロップしました


### clean directories

In [3]:
def clean_directories():
    # 削除対象のディレクトリ
    target_dirs = ['videos', 'images', 'images_cropped']
    
    for dir_name in target_dirs:
        if os.path.exists(dir_name):
            # ディレクトリ内のファイルをすべて削除
            for file in os.listdir(dir_name):
                file_path = os.path.join(dir_name, file)
                try:
                    if os.path.isfile(file_path):
                        os.unlink(file_path)
                    elif os.path.isdir(file_path):
                        shutil.rmtree(file_path)
                except Exception as e:
                    print(f'{file_path}の削除中にエラーが発生しました: {e}')
            print(f'{dir_name}ディレクトリを空にしました')
        else:
            print(f'{dir_name}ディレクトリは存在しません')

# クリーン処理の実行
clean_directories()


videosディレクトリを空にしました
imagesディレクトリを空にしました
images_croppedディレクトリを空にしました
